In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import warnings
warnings.filterwarnings("ignore")

import pathlib
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as pyo


from src.download import download_experiment_data, download_recprice_data, download_order_data, download_tender_data
from src.metrics import calculate_absolute_metrics, get_switchback_results, calculate_ratio_metrics
from src.prepare import prepare_recprice_data, prepare_order_data, get_orders_with_recprice_df, get_hex
from src.visualization import plot_switches_matrix, plot_conversions_by_time, plot_prices_by_time
from src.visualization import plot_metric_by_time, plot_metric_by_hex

## SB Results: Bad Bids

- link

### Костыли

In [4]:
import pandas as pd
from pathlib import Path

def load_data(DATA_ROOT_PATH, file_prefix):
    """
    Загружает данные из CSV или Parquet файла
    
    Args:
        DATA_ROOT_PATH: путь к директории с данными
        file_prefix: префикс файла ('df_recprice', 'df_tenders' или 'df_orders')
    """
    csv_path = DATA_ROOT_PATH / f'{file_prefix}.csv'
    pqt_path = DATA_ROOT_PATH / f'{file_prefix}.pqt'
    
    # Проверяем наличие файлов
    csv_exists = csv_path.exists()
    pqt_exists = pqt_path.exists()
    
    if not csv_exists and not pqt_exists:
        raise FileNotFoundError(f"Не найдено ни одного файла для {file_prefix}! "
                              f"Проверьте наличие файлов:\n"
                              f"{csv_path}\n{pqt_path}")
    
    # Определяем колонки с датами в зависимости от типа файла
    if file_prefix == 'df_recprice':
        date_columns = ['utc_recprice_dttm', 'local_recprice_dttm']
    else:
        date_columns = ['utc_order_dttm', 'local_order_dttm']
    
    # Если есть только один файл
    if csv_exists and not pqt_exists:
        print(f"Найден только CSV файл для {file_prefix}")
        df = pd.read_csv(csv_path)
        # Преобразуем все даты в datetime
        for col in date_columns:
            if col in df.columns and not pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = pd.to_datetime(df[col])
        print(f"\nCSV файл {file_prefix}:")
        print(f"Строк: {len(df)}")
        print(f"Даты: с {df[date_columns[0]].min()} по {df[date_columns[0]].max()}")
        return df
    
    if pqt_exists and not csv_exists:
        print(f"Найден только Parquet файл для {file_prefix}")
        df = pd.read_parquet(pqt_path)
        # Преобразуем все даты в datetime
        for col in date_columns:
            if col in df.columns and not pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = pd.to_datetime(df[col])
        print(f"\nParquet файл {file_prefix}:")
        print(f"Строк: {len(df)}")
        print(f"Даты: с {df[date_columns[0]].min()} по {df[date_columns[0]].max()}")
        return df
    
    # Если есть оба файла - оригинальная логика
    print(f"Найдены оба файла для {file_prefix}, сравниваем даты...")
    
    # Загружаем оба файла
    df_csv = pd.read_csv(csv_path)
    df_pqt = pd.read_parquet(pqt_path)
    
    # Преобразуем все даты в datetime для обоих датафреймов
    for col in date_columns:
        if col in df_csv.columns and not pd.api.types.is_datetime64_any_dtype(df_csv[col]):
            df_csv[col] = pd.to_datetime(df_csv[col])
        if col in df_pqt.columns and not pd.api.types.is_datetime64_any_dtype(df_pqt[col]):
            df_pqt[col] = pd.to_datetime(df_pqt[col])
    
    # Приводим даты к единому формату (без временной зоны)
    for col in date_columns:
        if col in df_csv.columns and df_csv[col].dt.tz is not None:
            df_csv[col] = df_csv[col].dt.tz_localize(None)
        if col in df_pqt.columns and df_pqt[col].dt.tz is not None:
            df_pqt[col] = df_pqt[col].dt.tz_localize(None)
    
    # Получаем максимальные даты
    max_date_csv = df_csv[date_columns[0]].max()
    max_date_pqt = df_pqt[date_columns[0]].max()
    
    print(f"\nCSV файл {file_prefix}:")
    print(f"Строк: {len(df_csv)}")
    print(f"Даты: с {df_csv[date_columns[0]].min()} по {max_date_csv}")
    
    print(f"\nParquet файл {file_prefix}:")
    print(f"Строк: {len(df_pqt)}")
    print(f"Даты: с {df_pqt[date_columns[0]].min()} по {max_date_pqt}")
    
    # Находим пересечение дат
    csv_dates = set(df_csv[date_columns[0]].dt.date)
    pqt_dates = set(df_pqt[date_columns[0]].dt.date)
    common_dates = csv_dates.intersection(pqt_dates)
    
    # Подсчитываем количество строк в период пересечения
    csv_common_rows = df_csv[df_csv[date_columns[0]].dt.date.isin(common_dates)].shape[0]
    pqt_common_rows = df_pqt[df_pqt[date_columns[0]].dt.date.isin(common_dates)].shape[0]
    
    print(f"\nПериод пересечения дат:")
    print(f"Количество дней: {len(common_dates)}")
    print(f"Количество строк в CSV: {csv_common_rows}")
    print(f"Количество строк в Parquet: {pqt_common_rows}")
    
    # Выбираем файл с более поздней датой
    if max_date_csv > max_date_pqt:
        print("\nВыбран CSV файл (более поздняя дата)")
        return df_csv
    else:
        print("\nВыбран Parquet файл (более поздняя дата)")
        return df_pqt

### Parameters

__Mutable__

In [3]:
EXP_ID = 2763
USER_NAME = 'nusuev_sb'+str(EXP_ID)

ORDER_TYPE = 'auto_econom'
ORDER_TYPE_ID = 1

DAYS_BEFORE = 0

__Immutable__

In [6]:
DATA_ROOT_PATH = pathlib.Path(f'data/exp_id={EXP_ID}')
if not DATA_ROOT_PATH.exists():
    DATA_ROOT_PATH.mkdir(parents=True, exist_ok=True)
    

PLOT_ROOT_PATH = pathlib.Path(f'data/exp_id={EXP_ID}/plots')
if not PLOT_ROOT_PATH.exists():
    PLOT_ROOT_PATH.mkdir(parents=True, exist_ok=True)

### Experiment Data

__Download__

In [7]:
df_exp = download_experiment_data(exp_id=EXP_ID, user_name=USER_NAME)

df_exp['hour'] = df_exp['switch_start_dttm'].dt.hour
df_exp['hour'] = df_exp['hour'].astype('category')
df_exp['weekday_name'] = df_exp['switch_start_dttm'].dt.day_name()
df_exp['weekday_name'] = df_exp['weekday_name'].astype('category')

df_exp.to_parquet(DATA_ROOT_PATH / 'df_exp.pqt')

EXP_START_DATE = df_exp.utc_start_dttm.dt.date.astype('str').iloc[0]
EXP_STOP_DATE = df_exp.utc_finish_dttm.dt.date.astype('str').iloc[0]
BEFORE_START_DATE = (df_exp.utc_start_dttm.dt.date - timedelta(days=DAYS_BEFORE)).astype('str').iloc[0]
CITY_ID = df_exp.city_id.iloc[0]
EXP_NAME = df_exp.exp_name.iloc[0]

print(
    f"""
    before_start_date: {BEFORE_START_DATE}
    exp_start_date: {EXP_START_DATE}
    exp_stop_date: {EXP_STOP_DATE}
    city_id: {CITY_ID}
    exp_name: {EXP_NAME}
    """
)


    before_start_date: 2025-02-28
    exp_start_date: 2025-02-28
    exp_stop_date: 2025-03-28
    city_id: 4515
    exp_name: (re-) [4515, Puerto Vallarta] Bad Bids v0
    


__Validity__

Switch Splitting. Total.

In [ ]:
df_exp.groupby('group_name').size()

Switch Splitting. By week day and hour.

In [13]:
def plot_switches_matrix(df_exp, plot_root_path, is_show=True):
    
    pyo.init_notebook_mode()
    
    gb = (df_exp
          .groupby(['weekday_name', 'hour', 'group_name'])
          .size()
          .rename('switches_count')
          .reset_index()
          .pivot(index=['weekday_name', 'hour'], columns='group_name', values='switches_count')
          .reset_index())
    gb['abs_diff'] = gb['A'] - gb['Control']
    gb = gb.pivot(index='hour', columns='weekday_name', values='abs_diff')
    gb = gb[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
    
    fig = px.imshow(gb, labels=dict(x="Day of Week", y="Hour of Day", color='A - Control'))
    fig.write_html(plot_root_path / 'switches_matrix.html')
    
    if is_show:
        fig.show()

In [ ]:
plot_switches_matrix(
    df_exp,
    plot_root_path=PLOT_ROOT_PATH,
    is_show=True,
)

### Metrics Data

__Recprice__

In [ ]:
df_recprice = download_recprice_data(
    start_date=BEFORE_START_DATE,
    stop_date=EXP_STOP_DATE,
    city_id=CITY_ID,
    user_name=USER_NAME,
    printBool=True
)
df_recprice.to_parquet(DATA_ROOT_PATH / 'df_recprice.pqt')

In [ ]:
df_recprice = load_data(DATA_ROOT_PATH, 'df_recprice')
df_recprice_prepared = prepare_recprice_data(df_recprice)
df_recprice_prepared.to_parquet(DATA_ROOT_PATH / 'df_recprice_prepared.pqt')

__Tenders__

In [8]:
df_tenders = download_tender_data(
    start_date=BEFORE_START_DATE,
    stop_date=EXP_STOP_DATE,
    city_id=CITY_ID,
    user_name=USER_NAME,
    printBool=True
)
df_tenders.to_parquet(DATA_ROOT_PATH / 'df_tenders.pqt')


    WITH incity AS (SELECT *
                    FROM (SELECT city_id                                                                  AS city_id,
                                order_type                                                                AS order_type,
                                order_uuid                                                                AS order_uuid,
                                user_id                                                                   AS user_id,
                                order_timestamp                                                           AS local_order_dttm,
                                TIMESTAMP(FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', order_timestamp),
                                        timezone)                                                         AS utc_order_dttm,
                                tender_uuid                                                               AS tender_uuid,
                    

KeyboardInterrupt: 

In [ ]:
df_tenders = load_data(DATA_ROOT_PATH, 'df_tenders')
df_tenders_prepared = prepare_order_data(df_tenders)
df_tenders_prepared.to_parquet(DATA_ROOT_PATH / 'df_tenders_prepared.pqt')

__Orders (with recprice)__

In [ ]:
df_orders = download_order_data(
    start_date=BEFORE_START_DATE,
    stop_date=EXP_STOP_DATE,
    city_id=CITY_ID,
    user_name=USER_NAME,
    printBool=True
)
df_orders.to_parquet(DATA_ROOT_PATH / 'df_orders.pqt')

In [21]:
df_orders = load_data(DATA_ROOT_PATH, 'df_orders')
df_orders_prepared = prepare_order_data(df_orders)
df_orders_prepared.to_parquet(DATA_ROOT_PATH / 'df_orders_prepared.pqt')

In [ ]:
df_orders_with_recprice = get_orders_with_recprice_df(df_orders_prepared, df_recprice_prepared)
df_orders_with_recprice['group_name'] = df_orders_with_recprice['recprice_group_name']
df_orders_with_recprice.to_parquet(DATA_ROOT_PATH / 'df_orders_with_recprice.pqt')

### Results

In [ ]:
# EXP_ID = 2574
# DATA_ROOT_PATH = pathlib.Path(f'data/exp_id={EXP_ID}')

# df_recprice_prepared = pd.read_parquet(DATA_ROOT_PATH / 'df_recprice_prepared.pqt')
# df_orders_with_recprice = pd.read_parquet(DATA_ROOT_PATH / 'df_orders_with_recprice.pqt')

__Total__

In [ ]:
df_metrics_total = calculate_absolute_metrics(
    df_recprice_prepared,
    df_orders_with_recprice,
    group_cols=['group_name', 'switch_start_dttm', 'switch_finish_dttm'],
)

metrics_total_tbl = get_switchback_results(df_metrics_total, alpha=0.05)[
    ['metric', 'control_value', 'experimental_value', 'uplift_rel', 'pvalue', 'is_significant']
]

metrics_total_tbl.head()

__By segment__

In [ ]:
condition_on_recprice = df_recprice_prepared.order_type_id == ORDER_TYPE_ID
condition_on_orders = df_orders_with_recprice.order_type == ORDER_TYPE

df_metrics_total = calculate_absolute_metrics(
    df_recprice_prepared[condition_on_recprice].copy(),
    df_orders_with_recprice[condition_on_orders].copy(),
    group_cols=['group_name', 'switch_start_dttm', 'switch_finish_dttm'],
)

metrics_total_tbl = get_switchback_results(df_metrics_total, alpha=0.05)[
    ['metric', 'control_value', 'experimental_value', 'uplift_rel', 'pvalue', 'is_significant']
]

metrics_total_tbl.head()

### Metrics by time

In [ ]:
GROUP_COLS = ['group_name', 'time']

In [ ]:
# Calculate Absolute Metrics
df_metrics_grouped = calculate_absolute_metrics(
    df_recprice_prepared,
    df_orders_with_recprice,
    group_cols=GROUP_COLS,
)

# Calculate Ratio Metrics
df_metrics_grouped = calculate_ratio_metrics(df_metrics_grouped)

__Conversions__

In [ ]:
plot_conversions_by_time(
    df_metrics_grouped,
    grouped_column='time',
    plot_root_path=PLOT_ROOT_PATH,
    is_before=True,
    is_show=True,
)

In [ ]:
plot_prices_by_time(
    df_metrics_grouped,
    grouped_column='time',
    plot_root_path=PLOT_ROOT_PATH,
    is_before=True,
    is_show=True,
)

In [ ]:
plot_metric_by_time(
    df_metrics_grouped,
    grouped_column='time',
    metric_name='surge',
    plot_root_path=PLOT_ROOT_PATH,
    is_before=True,
    is_show=True,
)

### Metrics Histogram

In [ ]:
sns.histplot(
    df_recprice_prepared,
    x='surge',
    hue='group_name',
    hue_order=['Control', 'A'],
    bins=np.arange(0.85, 2.85, 0.02),
).set_title('Total');

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='analytics-dev-333113')

query = f"""
DECLARE start_date DATE;
SET start_date = DATE '2024-01-01';

select DATE_SUB(start_date, INTERVAL 1 DAY);
    """
    
client.query(query).to_dataframe()

# side note

In [25]:
df_orders_with_recprice = pd.read_parquet('data/exp_id=2640/df_orders_with_recprice.pqt')

In [ ]:
df_orders_with_recprice = df_orders_with_recprice[df_orders_with_recprice.order_type == ORDER_TYPE]
df_orders_with_recprice.head().transpose()

In [28]:
df_recprice_prepared = pd.read_parquet('data/exp_id=2640/df_recprice_prepared.pqt')

In [ ]:
df_recprice_prepared = df_recprice_prepared[df_recprice_prepared.order_type_id == ORDER_TYPE_ID]
df_recprice_prepared.head().transpose()

In [ ]:
import src.metrics as metrics

def calculate_absolute_metrics(df_recprice, df_order_with_recprice, group_cols):
    dfm = (metrics.metric_accepted_orders_count(df_order_with_recprice, group_cols)
           .merge(metrics.metric_bids_count(df_order_with_recprice, group_cols), on=group_cols, how='left'))
    return dfm


df_metrics_total = calculate_absolute_metrics(
    df_recprice_prepared,
    df_orders_with_recprice,
    group_cols=['order_uuid', 'group_name'],
)

df_metrics_total['acceptance_rate'] = df_metrics_total['accepted_orders_count'] / df_metrics_total['bids_count']

df_metrics_total

In [ ]:
import seaborn as sns
import numpy as np

df_metrics_total = calculate_absolute_metrics(
    df_recprice_prepared,
    df_orders_with_recprice,
    group_cols=['order_uuid', 'group_name'],
)

df_metrics_total['acceptance_rate'] = df_metrics_total['accepted_orders_count'] / df_metrics_total['bids_count']
df_metrics_total = df_metrics_total[df_metrics_total['bids_count'] > 0]

sns.histplot(
    df_metrics_total,
    x='acceptance_rate',
    hue='group_name',
    hue_order=['Control', 'GroupA'],
    bins=np.arange(0.0, 1.0, 0.1),
).set_title('Total');

In [ ]:
df_metrics_total[df_metrics_total['group_name'] == 'Control']['acceptance_rate'].describe()


In [ ]:
df_metrics_total[df_metrics_total['group_name'] == 'GroupA']['acceptance_rate'].describe()

In [83]:
def linearize_data(data, control, numerator, denominator):
    df = data.copy()
    conversions = df[df["group_name"] == control][numerator].sum()
    visitors = df[df["group_name"] == control][denominator].sum()
    k = conversions / visitors
    df[f"linearized_{denominator}"] = k * df[denominator]
    df[f"linearized_{numerator}"] = (
        df[numerator] - df[f"linearized_{denominator}"]
        ).astype(float)
    return df

In [ ]:
df_metrics_total = calculate_absolute_metrics(
    df_recprice_prepared,
    df_orders_with_recprice,
    group_cols=['order_uuid', 'group_name'],
)

df_metrics_total = df_metrics_total[df_metrics_total['bids_count'] > 0]

lin_metrics = linearize_data(df_metrics_total, 'Control', 'accepted_orders_count', 'bids_count')

lin_metrics['acceptance_rate'] = lin_metrics['linearized_accepted_orders_count'] / lin_metrics['linearized_bids_count']

sns.histplot(
    lin_metrics,
    x='acceptance_rate',
    hue='group_name',
    hue_order=['Control', 'GroupA'],
    bins=np.arange(0.0, 12.0, 0.1),
).set_title('Total');

In [ ]:
# Группируем данные по acceptance_rate и group_name, считаем количество
plot_data = lin_metrics.groupby(['acceptance_rate', 'group_name']).size().reset_index(name='count')

# Создаем график
plt.figure(figsize=(10, 6))

# Строим линии для каждой группы
for group in ['Control', 'GroupA']:
    group_data = plot_data[plot_data['group_name'] == group]
    plt.plot(group_data['acceptance_rate'], group_data['count'], 
             marker='o', linestyle='-', label=group)

plt.xlabel('Acceptance Rate')
plt.ylabel('Count')
plt.title('Distribution of Acceptance Rate by Group')
plt.legend()
plt.grid(True)